In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
sns.set_context('poster', font_scale=1.25)

# A single deme (well-mixed test tube)

In [ ]:
num_individuals=500
numAlleles = num_individuals
numGenerations = 100

ind_list = np.array([re.Individual(i) for i in np.random.randint(low=0, high=numAlleles, size=num_individuals)])
d = re.Deme(numAlleles, ind_list, 0)

frac_gen, history = re.simulate_deme(d, numGenerations)

# Checking that the allele frequency is what is predicted

print 'Allele frequency is correct:' , d.check_allele_frequency()

plt.plot(frac_gen, history)